# Intro


Lately, I've been following [FastHTML](https://about.fastht.ml/) from a distance. As someone who sticks to backend Python development, frontend development has always been a bit foreign to me, but I'm interested in giving it a shot. FastHTML feels like a good way to get started with some basics by building small apps.

I've also noticed a lot of chatter on X about [Colpali](https://github.com/illuin-tech/colpali) and document retrieval with vision language models, which caught my attention. I like exploring new stuff so I want to see what that is all about.

On top of that, I'm still enjoying [Modal](https://modal.com/), which I’ve written about before [here](https://drchrislevy.github.io/posts/modal_fun/modal_blog.html) and [here](https://drchrislevy.github.io/posts/intro_modal/intro_modal.html). I thought it would be fun to combine these tools into a simple app and see what I can learn from it.


# ColPali

There are already so many great resources out there about ColPali. Checkout the resources [below](#resources) for more information.
I will give a quick overview.

I have already deployed ColPali to Modal as a remote function I can call, running on an A10 GPU.

```
modal deploy pdf_retriever.py
```

Remember that with Modal, you only pay for compute when running requests in active containers. My deployed app
can sit there idle without costing me anything!

![ColPali Model Deployed on Modal](imgs/colpali_modal.png)


There are a couple functions I have decorated with `@modal.method()` within the `PDFRetriever` class:

**TODO: add link to the code**

-  `def forward(self, inputs)`
-  `def top_pages(self, pdf_url, queries, use_cache=True, top_k=1)`


Let's look at the `forward` function first as it can be used to run inference on a list of strings or images to get the embeddings.

First we will pass in text inputs to ColPali.

In [1]:
# | warning: false
import modal

get_embeddings = modal.Function.lookup("pdf-retriever", "PDFRetriever.forward")
embeddings_batch = get_embeddings.remote(["What days in October had anomalies in the sales data?"])
assert len(embeddings_batch) == 1  # we passed in one document i.e. batch size of 1
embeddings = embeddings_batch[0]
print(embeddings.shape)
embeddings

/Users/christopher/personal_projects/DrChrisLevy.github.io/posts/colpali/env/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([25, 128])


tensor([[ 0.1680, -0.0111,  0.0957,  ..., -0.0272, -0.0762, -0.0249],
        [-0.0737, -0.0918,  0.0698,  ...,  0.0195, -0.1162,  0.0593],
        [ 0.1357, -0.0054, -0.0986,  ...,  0.1279, -0.0537, -0.0309],
        ...,
        [ 0.1270, -0.0297,  0.0503,  ...,  0.0908, -0.1523,  0.0200],
        [ 0.1152, -0.0654,  0.0454,  ...,  0.0996, -0.1079, -0.0101],
        [ 0.1533,  0.0125, -0.0135,  ...,  0.1553, -0.0972, -0.0305]],
       dtype=torch.bfloat16)

The first thing to note is that we don't get a single dense embedding vector.
Traditionally that is the case where a single vector is used to represent one input.
But ColPali is generating ColBERT-style multi-vector representations of the input.
With the late interaction paradigm you get back multiple embeddings, one per input **token**.
Each embedding is 128-dimensional. 


ColPali is trained to take image documents as input.
It was trained on query-document pairs where each document is a page of a PDF.
Each PDF page ("document") is treated as an image. It uses a vision language model to create 
multi-vector embeddings purely from visual document features.

Consider the following image of a PDF page from the ColPali paper:

![ColPali Paper PDF Page 2](imgs/colpali_paper_page_sample.png)
We can pass this image to the `forward` function and get the embeddings back.
The ColPali model divides each page image into a 32 x 32 = 1024 patches.
In addition to the image grid patches, ColPali includes 6 instruction text tokens that are prepended to the image input. 
These tokens represent the text: "Describe the image." Combining the image grid patches and the instruction tokens, we get:
1024 (image patches) + 6 (instruction tokens) = 1030 total patches/embeddings.




In [2]:
from PIL import Image

img = Image.open("imgs/colpali_paper_page_sample.png")
embeddings = get_embeddings.remote([img])[0]
print(embeddings.shape)
embeddings

torch.Size([1030, 128])


tensor([[-0.1543, -0.0332, -0.1001,  ...,  0.1436, -0.0928,  0.1108],
        [-0.1152,  0.0593,  0.0972,  ..., -0.0347, -0.0114,  0.0815],
        [-0.1533,  0.0422,  0.0776,  ..., -0.0248, -0.0116,  0.0518],
        ...,
        [ 0.0884,  0.0054,  0.0698,  ..., -0.0850, -0.1206, -0.0461],
        [-0.0116,  0.0908,  0.0752,  ..., -0.0248, -0.0449, -0.1016],
        [ 0.0116,  0.0442,  0.1416,  ...,  0.0087, -0.1602, -0.1611]],
       dtype=torch.bfloat16)

Each PDF page/image (document) can be indexed with the ColPali model to get the multi-vector embeddings per page.
At query time, we use the same model to generate multi-vector embeddings for the query. 
So both queries and documents are represented as sets of vectors rather than single vector.

The MaxSim (Maximum Similarity) scoring function is used to compute the similarity between query embeddings and document embeddings.
The scoring function performs the following steps:

- Computes dot products between all query token embeddings and all document page patch embeddings
- Applies a max reduce operation over the patch dimension
- Performs a sum reduce operation over the query tokens

There is a great and simple explanation in this [blog post](Both queries and documents are represented as sets of vectors rather than single vector.)

I have wrapped the logic for a given pdf url and query/question within the deployed Modal function `def top_pages(self, pdf_url, queries, use_cache=True, top_k=1)`.

In [3]:
get_top_pages = modal.Function.lookup("pdf-retriever", "PDFRetriever.top_pages")
pdf_url = "https://arxiv.org/pdf/2407.01449"
top_pages = get_top_pages.remote(pdf_url, queries=["How does the latency between ColPali and standard retrieval methods compare?"], top_k=5)[0]
top_pages

[1, 0, 4, 5, 13]

The function takes a `pdf_url` and a list of `queries` (questions) and returns the top `top_k` pages for each query/question.
ColPali is used to retrieve the most relevant pages from the PDF. 

# Generating the Final Answer with a Vision Language Model

Once we have the top pages/images as context, we can pass them along with the query/question to a vision language model to generate an answer.
The images are passed as the context and the question/query is passed as text. I have this logic deployed in a Modal Application as well
running on CPU. It communicates with the deployed ColPali Modal app running on the GPU when it needs to compute the embeddings.

```
modal deploy multi_modal_rag.py
```

The deployed Modal function here is 

```
def answer_question_with_image_context(pdf_url, query, top_k=1, use_cache=True, max_new_tokens=2000, additional_instructions=""):
```

**TODO: add link to the code**

I will explain all the arguments in the function later when we look at the FastHTML App.




In [9]:
answer_question_with_image_context = modal.Function.lookup("multi-modal-rag", "answer_question_with_image_context")
res = answer_question_with_image_context.remote_gen(
    pdf_url="https://arxiv.org/pdf/2407.01449", query="How does the latency between ColPali and standard retrieval methods compare?", top_k=5
)
answer = "".join([chunk for chunk in res if type(chunk) == str])
print(answer)

The latency comparison between ColPali and standard retrieval methods shows a significant improvement. 

- **Standard Retrieval**: The latency is approximately **7.22 seconds per page** for processing.
- **ColPali**: The latency is reduced to about **0.39 seconds per page**.

Additionally, when querying, ColPali has a latency of **22 milliseconds per query**, while standard methods require longer processing times. This indicates that ColPali is considerably faster than traditional methods for both indexing and querying.


I am simply using OpenAI's `gpt-4o-mini` as the vision language model here.

# FastHTML App

# Resources

In no particular order:

- [Colpali paper](https://arxiv.org/pdf/2407.01449v2)
- [Colbert paper](https://arxiv.org/pdf/2004.12832)
- [Colbert V2 paper](https://arxiv.org/pdf/2112.01488)
- [PaliGemma](https://arxiv.org/pdf/2407.07726)
- [A little pooling goes a long way for multi-vector representations: Blog answer.ai](https://www.answer.ai/posts/colbert-pooling.html)
    - [Reducing the Footprint of Multi-Vector Retrieval with Minimal Performance Impact via Token Pooling, Paper](https://arxiv.org/pdf/2409.14683)
- [PLAID paper](https://arxiv.org/pdf/2205.09707)
- [Beyond the Basics of Retrieval for Augmenting Generation (w/ Ben Clavié), Youtube Talk](https://www.youtube.com/watch?v=0nA5QG3087g)
- [RAG is more than dense embedding, Google Slides, Ben Clavié](https://docs.google.com/presentation/d/1Zczs5Sk3FsCO06ZLDznqkOOhbTe96PwJa4_7FwyMBrA/edit#slide=id.p)
- The quick start in the README [Original ColPali Repo](https://github.com/illuin-tech/colpali) as well as the sample [inference code](https://github.com/illuin-tech/colpali/blob/main/scripts/infer/run_inference_with_python.py)
- [Hugging Face Model Cards](https://huggingface.co/vidore/colpali-v1.2)
- [The Future of Search: Vision Models and the Rise of Multi-Model Retrieval](https://mcplusa.com/the-future-of-search-vision-models-and-the-rise-of-multi-model-retrieval/)
- [Scaling ColPali to billions of PDFs with Vespa](https://blog.vespa.ai/scaling-colpali-to-billions/)
- [Beyond Text: The Rise of Vision-Driven Document Retrieval for RAG](https://blog.vespa.ai/the-rise-of-vision-driven-document-retrieval-for-rag/)
- [Vision Language Models Explained](https://huggingface.co/blog/vlms)
- [Document Similarity Search with ColPali](https://huggingface.co/blog/fsommers/document-similarity-colpali)
- [Jo Kristian Bergum: X](https://x.com/jobergum)
- [Manuel Faysse: X](https://x.com/ManuelFaysse)
- [Tony Wu: X](https://x.com/tonywu_71)
- [Omar Khattab: X](https://x.com/lateinteraction?lang=en)
- [fastHTML](https://about.fastht.ml/)